In [4]:
##%overwritefile
##%file:src/compile_out_file.py
##%noruncode
    def getCompout_filename(self,cflags,outfileflag,defoutfile):
        outfile=''
        binary_filename=defoutfile
        index=0
        for s in cflags:
            if s.startswith(outfileflag):
                if(len(s)>len(outfileflag)):
                    outfile=s[len(outfileflag):]
                    del cflags[index]
                else:
                    outfile=cflags[cflags.index(outfileflag)+1]
                    if outfile.startswith('-'):
                        outfile=binary_filename
                    del cflags[cflags.index(outfileflag)+1]
                    del cflags[cflags.index(outfileflag)]
            binary_filename=outfile
            index+=1
        return binary_filename


[MyPython] Info:file h:\Jupyter\Myjupyter-kernel\kernel\Vimscript\src/compile_out_file.py created successfully


In [5]:
##%overwritefile
##%file:src/compile_with_sc.py
##%noruncode
    def compile_with_sc(self, source_filename, binary_filename, cflags=None, ldflags=None,env=None,magics=None):
        outfile=binary_filename
        orig_cflags=cflags
        orig_ldflags=ldflags
        ccmd=[]
        clargs=[]
        crargs=[]
        outfileflag=[]
        oft=''
        if len(self.kernel_info['compiler']['outfileflag'])>0:
            oft=self.kernel_info['compiler']['outfileflag']
            outfileflag=[oft]
            binary_filename=self.getCompout_filename(cflags,oft,outfile)
        args=[]
        if magics!=None and len(self.addkey2dict(magics,'ccompiler'))>0:
            ## use code line ccompiler lable
            args = magics['ccompiler'] + orig_cflags +[source_filename] + orig_ldflags
        else:
            ## use kernel default compiler -> kernel_info['compiler']['cmd']
            if len(self.kernel_info['compiler']['cmd'])>0:
                ccmd+=[self.kernel_info['compiler']['cmd']]
            if len(self.kernel_info['compiler']['clargs'])>0:
                clargs+=self.kernel_info['compiler']['clargs']
            if len(self.kernel_info['compiler']['crargs'])>0:
                crargs+=self.kernel_info['compiler']['crargs']

            args = ccmd+cflags+[source_filename] +clargs+outfileflag+ [binary_filename]+crargs+ ldflags
        # self._log(''.join((' '+ str(s) for s in args))+"\n")
        return self.create_jupyter_subprocess(args,env=env,magics=magics),binary_filename,args


[MyPython] Info:file h:\Jupyter\Myjupyter-kernel\kernel\Vimscript\src/compile_with_sc.py created successfully


In [6]:
##%overwritefile
##%file:src/c_exec_sc_.py
##%noruncode
    def _exec_sc_(self,source_filename,magics):
        self._logln('Generating executable file')
        with self.new_temp_file(suffix=self.kernel_info['execsuffix']) as binary_file:
            magics['status']='compiling'
            p,outfile,tsccmd = self.compile_with_sc(
                source_filename, 
                binary_file.name,
                self.get_magicsSvalue(magics,'cflags'),
                self.get_magicsSvalue(magics,'ldflags'),
                self.get_magicsbykey(magics,'env'),
                magics=magics)
            returncode=p.wait_end(magics)
            p.write_contents()
            magics['status']=''
            binary_file.name=os.path.join(os.path.abspath(''),outfile)
            if returncode != 0:  
                ## Compilation failed
                self._logln(' '.join((str(s) for s in tsccmd))+"\n",3)
                self._logln("compiler exited with code {}, the executable will not be executed".format(returncode),3)
                ## delete source files before exit
                ## os.remove(source_filename)
                os.remove(binary_file.name)
        return p.returncode,binary_file.name


[MyPython] Info:file h:\Jupyter\Myjupyter-kernel\kernel\Vimscript\src/c_exec_sc_.py created successfully
